In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql import Column

In [2]:
sc=SparkContext(appName="pg_dataframe") 
spark=SparkSession.builder.appName('pg_dataframe').master("Yct201811021847").getOrCreate()

In [3]:
def create_df_from_postgres():
    df=spark.read.format('jdbc').options(
        url='jdbc:postgresql://localhost:5432/northwind',
        dbtable='public.orders',
        user='postgres',
        password='iamroot'
    ).load()
    return df

In [4]:
data=create_df_from_postgres()

In [5]:
data.show(2)

+-------+----------+----------+----------+------------+-----------+-------+-------+--------------------+------------------+--------+----------+--------------+-----------+
|orderid|customerid|employeeid| orderdate|requireddate|shippeddate|shipvia|freight|            shipname|       shipaddress|shipcity|shipregion|shippostalcode|shipcountry|
+-------+----------+----------+----------+------------+-----------+-------+-------+--------------------+------------------+--------+----------+--------------+-----------+
|  10248|     VINET|         5|1996-07-04|  1996-08-01| 1996-07-16|      3|  32.38|Vins et alcools C...|59 rue de l'Abbaye|   Reims|      null|         51100|     France|
|  10249|     TOMSP|         6|1996-07-05|  1996-08-16| 1996-07-10|      1|  11.61|  Toms Spezialitäten|     Luisenstr. 48| Münster|      null|         44087|    Germany|
+-------+----------+----------+----------+------------+-----------+-------+-------+--------------------+------------------+--------+----------+--

In [6]:
type(spark)

pyspark.sql.session.SparkSession

In [7]:
data.printSchema()

root
 |-- orderid: short (nullable = true)
 |-- customerid: string (nullable = true)
 |-- employeeid: short (nullable = true)
 |-- orderdate: date (nullable = true)
 |-- requireddate: date (nullable = true)
 |-- shippeddate: date (nullable = true)
 |-- shipvia: short (nullable = true)
 |-- freight: float (nullable = true)
 |-- shipname: string (nullable = true)
 |-- shipaddress: string (nullable = true)
 |-- shipcity: string (nullable = true)
 |-- shipregion: string (nullable = true)
 |-- shippostalcode: string (nullable = true)
 |-- shipcountry: string (nullable = true)



In [8]:
type(data)

pyspark.sql.dataframe.DataFrame

In [9]:
data.summary().show()

+-------+-----------------+----------+------------------+------------------+------------------+--------------------+--------------------+--------+----------+------------------+-----------+
|summary|          orderid|customerid|        employeeid|           shipvia|           freight|            shipname|         shipaddress|shipcity|shipregion|    shippostalcode|shipcountry|
+-------+-----------------+----------+------------------+------------------+------------------+--------------------+--------------------+--------+----------+------------------+-----------+
|  count|              830|       830|               830|               830|               830|                 830|                 830|     830|       323|               811|        830|
|   mean|          10662.5|      null| 4.403614457831325|2.0072289156626506|   78.244204877492|                null|                null|    null|      null|39975.067357512955|       null|
| stddev|239.7446558319914|      null|2.499647540082388

In [11]:
data.describe(['shipname','freight']).show()

+-------+--------------------+------------------+
|summary|            shipname|           freight|
+-------+--------------------+------------------+
|  count|                 830|               830|
|   mean|                null|   78.244204877492|
| stddev|                null|116.77929406259489|
|    min|Alfred's Futterkiste|              0.02|
|    max|       Wolski Zajazd|           1007.64|
+-------+--------------------+------------------+



In [55]:
cust_country=data.groupBy(['customerid','shipcountry'])

In [49]:
type(cust_country)

pyspark.sql.group.GroupedData

In [61]:
cust_country.agg({'shipcountry':'count','freight':'avg'}).orderBy(['count(shipcountry)'],ascending=False).show()

+----------+-----------+------------------+------------------+
|customerid|shipcountry|count(shipcountry)|      avg(freight)|
+----------+-----------+------------------+------------------+
|     SAVEA|        USA|                31|215.60322700008268|
|     ERNSH|    Austria|                30|206.84633391698202|
|     QUICK|    Germany|                28| 200.2010712495872|
|     FOLKO|     Sweden|                19|   88.320000121468|
|     HUNGO|    Ireland|                19|145.01262895684494|
|     RATTC|        USA|                18|118.56722169452243|
|     HILAA|  Venezuela|                18| 69.95333398713007|
|     BERGS|     Sweden|                18| 86.64000034332275|
|     BONAP|     France|                17| 79.87470626831055|
|     FRANK|    Germany|                15|  93.5626670519511|
|     LEHMS|    Germany|                15| 67.80199979146322|
|     WARTH|    Finland|                15| 54.83200004498164|
|     KOENE|    Germany|                14| 58.12000067

In [17]:
cust_country.avg().show()

+----------+-------+--------------------+------------+---------------+------------+------------------+
|customerid|freight|            shipname|avg(orderid)|avg(employeeid)|avg(shipvia)|      avg(freight)|
+----------+-------+--------------------+------------+---------------+------------+------------------+
|     BSBEV|  22.77|       B's Beverages|     10289.0|            7.0|         3.0|22.770000457763672|
|     RICSU| 130.79|  Richter Supermarkt|     10751.0|            3.0|         3.0| 130.7899932861328|
|     DRACD|  79.25|Drachenblut Delik...|     10825.0|            1.0|         1.0|             79.25|
|     RICSU|  30.85|  Richter Supermarkt|     10951.0|            9.0|         2.0|30.850000381469727|
|      null| 147.26|                null|     10294.0|            4.0|         2.0|147.25999450683594|
|      null|  17.68|Magazzini Aliment...|     10300.0|            2.0|         2.0| 17.68000030517578|
|     GODOS| 107.83|                null|     10303.0|            7.0|   

In [40]:
d1=data.cube(['customerid','freight','shipcountry'])